## Preliminary Data Confrontation

Preliminary EDA and cleaning was done on the two Kickstarter datasets, to determine how to combine them, or whether to pick one dataset over another.

In [ ]:
// Create a Spark Context
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

// Note: Initial cleaning step added so that "" and \ are not escaped in strings. This is because movie names with commas
// in strings were being delimited when brought in

// First dataset
var df = sqlContext
    .read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("quote", "\"")
    .option("escape", "\"")
    .load("ks-projects-201612.csv")

In [ ]:
// Second dataset
var df2 = sqlContext
    .read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("quote", "\"")
    .option("escape", "\"")
    .load("ks-projects-201801.csv")

In [ ]:
// Get the dimensions of both years
print("2016: ")
println((df.count(), df.columns.size))
print("2018: ")
println((df2.count(), df2.columns.size))

// 2016 has 323,760 rows and 17 columns
// 2018 has 378,661 rows and 15 columns

In [ ]:
// Look at the first 20 rows of 2016 dataframe

df.show()

// Looks like it is erroneously bringing in four columns (_c13, c_14, c_15, c_16) which are just nulls

In [ ]:
// Look at first 20 rows of 2018 dataframe

df2.show()
// No blank columns in this dataset

In [ ]:
// Look at data types of columns
df.dtypes

In [ ]:
df2.dtypes
// Looks like the 2018 dataset has more appropriate string types. The 2016 one has strings for deadline, goal, launched
// pledged, and usd pledged, which should be dates and numbers/doubles, as in the 2018 dataset. 

In [ ]:
// We need to check 2016 and 2018 for duplicates. We will do this by merging 2016 to 2018 on the ID. We will compare IDs,
// as well as other columns. We will then choose how we process these datasets - e.g. to union them, or if only 2018
// dataset is required.

// Change the name of all the columns in the 2016 dataset. Renaming the 2016 dataset so that we can compare both later
// to see what columns are duplicates.

// Also renaming the 'usd pledged' variable in 2018 with an underscore so it is easier to refer to in code

val df_renamed = 
    df.withColumnRenamed("ID ", "ID_2016")
    .withColumnRenamed("name ", "name_2016")
    .withColumnRenamed("category ", "category_2016")
    .withColumnRenamed("main_category ", "main_category_2016")
    .withColumnRenamed("currency ", "currency_2016")
    .withColumnRenamed("deadline ", "deadline_2016")
    .withColumnRenamed("goal ", "goal_2016")
    .withColumnRenamed("launched ", "launched_2016")
    .withColumnRenamed("pledged ", "pledged_2016")
    .withColumnRenamed("state ", "state_2016")
    .withColumnRenamed("backers ", "backers_2016")
    .withColumnRenamed("usd pledged ", "usd_pledged_2016")

val df2_renamed = df2.withColumnRenamed("usd pledged", "usd_pledged")

// Remove the original dataframes to save space.
df.unpersist()
df2.unpersist()


In [ ]:
// Before we do a merge, we need to check if the ID values in both datasets are unique.
// We'll do a group by and count the IDs in both datasets. If there are duplicates, they will show up on this list as having
// appeared two or more times.

df_renamed.groupBy("ID_2016").count()
  .filter($"count" >= 2)
  .show()

// Result: The 2016 dataframe has no duplicate IDs

In [ ]:
df2_renamed.groupBy("ID").count()
  .filter($"count" >= 2)
  .show()

// Result: The 2018 dataframe has no duplicate IDs

In [ ]:
// Since IDs are unique, we will do a left join to see how many 2018 IDs are in the 2016 file. 
// If there are no nulls in the join, then the 2018 file contains all IDs from the 2016 file

val joined_df = df_renamed.join(df2_renamed, $"ID_2016" === $"ID", "left_outer")

In [ ]:
// Count the number of rows in joined_df. Should have the same number of rows as the 2016 dataset, as it's a left join.
joined_df.count()
//Result: 323,750 is the count of the 2016 dataset, so the join was correct

In [ ]:
// Now check for 'ID' values that are NULL (i.e. didn't match)

joined_df.filter("ID is null").show
// Result: Returns an empty table. There are no nulls, so all IDs in the 2016 dataset are in 2018 also.

In [ ]:
// As a further check, check how many rows have all 2016 columns equal the 2018 columns. The 2016 columns are named with a
// '_2016' suffix. Do a count.

joined_df.filter("ID_2016 = ID and name_2016 = name and category_2016 = category and main_category_2016 = main_category and currency_2016 = currency and deadline_2016 = deadline and goal_2016 = goal and launched_2016 = launched and pledged_2016 = pledged and state_2016 = state and goal_2016 = goal and backers_2016 = backers and usd_pledged_2016 = usd_pledged").count()


In [ ]:
// There are relatively few records where all columns match.
// This may be a parsing issue due to the different data types between 2016 and 2018 columns.

// Instead, we'll check individual fields to get a taste of how name, category, main_category, currency, 
// and pledged/goal/backers don't match for the same ID.

In [ ]:
joined_df.filter("ID_2016 = ID and name_2016 = name").count()
// Most names match

In [ ]:
// What name records didn't match? Take a look at the top 20
joined_df.filter("ID_2016 = ID and name_2016 != name").select("ID", "name_2016", "name").show()
// Result: The names are the same. There are likely small differences in text being picked up. 

In [ ]:
joined_df.filter("ID_2016 = ID and category_2016 = category").count()
// Most category records match

In [ ]:
joined_df.filter("ID_2016 = ID and main_category_2016 = main_category").count()
// Most main_category records match

In [ ]:
joined_df.filter("ID_2016 = ID and currency_2016 = currency").count()
// Most currency records match

In [ ]:
joined_df.filter("ID_2016 = ID and goal_2016 = goal").count()
// Most goal records match

In [ ]:
joined_df.filter("ID_2016 = ID and backers_2016 = backers").count()

In [ ]:
joined_df.filter("ID_2016 = ID and backers_2016 != backers").select("ID", "backers_2016", "state_2016", "backers").show()
// The backers numbers look to have legitimate differences. However, all the numbers in the sample 
// come from records that were 'live' in 2016 (i.e. ongoing kickstarters), so the 2018 data would 
// represent legitimate, updated backer information.

In [ ]:
// Confirming all instances of backers in 2018 being greater than 2016 are due to the 2016 campaigns being live at the time:
joined_df.filter("ID_2016 = ID and backers_2016 < backers").groupBy("state_2016").count()
  .show()

In [ ]:
joined_df.filter("ID_2016 = ID and usd_pledged_2016 = usd_pledged").count()

In [ ]:
joined_df.filter("ID_2016 = ID and usd_pledged_2016 != usd_pledged").select("ID", "usd_pledged_2016", "usd_pledged").show()
// 2018 figures are rounded to the nearest cent which causes the difference. Nearest cent is preferable given this is
// dollar values

We can conclude that the 2018 dataset includes all values from the 2016 dataset. Justifications: 
    1) It has all the IDs that the 2016 dataset has. 
    2) All the columns are assigned the correct data type when imported in 2018, but not for 2016. 
    3) 2018 has the same column values as 2016 for those records that are shared
    4) It has updated 'backer' counts for those campaigns that were 'live' in 2016, 
    5) It has correctly assigned decimals, rounded to two places, for the usd_pledged variable

Therefore, we will use the 2018 dataset exclusively as an input into the cleaning pipeline.